In [1]:
! wget http://imagine.enpc.fr/~shenx/data/Watermark.zip

--2023-03-13 20:42:06--  http://imagine.enpc.fr/~shenx/data/Watermark.zip
Resolving imagine.enpc.fr (imagine.enpc.fr)... 195.221.193.223
Connecting to imagine.enpc.fr (imagine.enpc.fr)|195.221.193.223|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423757918 (404M) [application/zip]
Saving to: ‘Watermark.zip’

Watermark.zip       100%[===================>] 404.13M  15.2MB/s    in 28s     

2023-03-13 20:42:36 (14.3 MB/s) - ‘Watermark.zip’ saved [423757918/423757918]



In [2]:
!unzip Watermark.zip

Streaming output truncated to the last 5000 lines.
  inflating: watermark/briquet_synthetic/briquet_8117.png  
  inflating: watermark/briquet_synthetic/briquet_8118.png  
  inflating: watermark/briquet_synthetic/briquet_8119.png  
  inflating: watermark/briquet_synthetic/briquet_812.png  
  inflating: watermark/briquet_synthetic/briquet_8120.png  
  inflating: watermark/briquet_synthetic/briquet_8121.png  
  inflating: watermark/briquet_synthetic/briquet_8122.png  
  inflating: watermark/briquet_synthetic/briquet_8123.png  
  inflating: watermark/briquet_synthetic/briquet_8124.png  
  inflating: watermark/briquet_synthetic/briquet_8125.png  
  inflating: watermark/briquet_synthetic/briquet_8126.png  
  inflating: watermark/briquet_synthetic/briquet_8127.png  
  inflating: watermark/briquet_synthetic/briquet_8128.png  
  inflating: watermark/briquet_synthetic/briquet_8129.png  
  inflating: watermark/briquet_synthetic/briquet_813.png  
  inflating: watermark/briquet_synthetic/briquet_81

In [3]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [4]:
def read_image(filename, byteorder='>'):
    img = cv2.imread(filename)    
    return img

def euclidean_distance(vects):
    x, y = vects
    return torch.sqrt(torch.sum(torch.square(x - y), dim=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return torch.mean((1 - y_true) * torch.square(y_pred) + y_true * torch.square(torch.clamp(margin - y_pred, min=0.0)))


def compute_accuracy(preds, labels, thr = 0.5):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    predictions = preds.clone()
    predictions[predictions <= thr] = 0
    predictions[predictions > thr] = 1
    return (predictions == labels).float().mean()



def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return torch.mean(y_true == y_pred)

In [5]:
import numpy as np
import statistics
def Preprocess(image):
    mean = np.array([204.5945,217.6841,217.6841])
    image[image<=mean]=0
    return image

In [6]:
def contrastive_loss_function(y_true, dist, margin=1.0):
    # euclidean distance

    dist_sq = torch.sum(torch.pow(diff, 2), 1)
    dist = torch.sqrt(dist_sq)
    mdist = margin - dist
    dist = torch.clamp(mdist, min=0.0)
    loss = y * dist_sq + (1 - y) * torch.pow(dist, 2)
    loss = torch.sum(loss) / 2.0 / x0.size()[0]
    return loss

In [7]:
from glob import glob

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self,folders, total_sample_size, resize = None, augment = False):
        #self.size = size
        self.folders = folders
        self.n_folders = len(folders)
        self.total_sample_size = total_sample_size
        self.augment = augment
        self.resize = resize
        
        self.pos_neg_map = torch.randint(0, 2, (total_sample_size,))
        
    def __len__(self):
        return self.total_sample_size

    def __getitem__(self, idx):
        label = self.pos_neg_map[idx]
        
        #for similar image
        if label == 0:
            folder = np.random.choice(folders, 1, replace=False).item()

            images = glob(os.path.join(folder , "*"))
            image1, image2 = np.random.choice(images, 2, replace=False)
            
        #for different image
        else:
            folder1, folder2 = np.random.choice(folders, 2, replace=False)
            image1 = np.random.choice(glob(os.path.join(folder1 , "*")), 1, replace=False).item()
            image2 = np.random.choice(glob(os.path.join(folder2 , "*")), 1, replace=False).item()
        
        img1 = cv2.imread(image1)
        img2 = cv2.imread(image2)
        
        if self.augment:
            img1 = Preprocess(img1)
            img2 = Preprocess(img2)
        
        if self.resize is not None:
            img1 = cv2.resize(img1, resize)
            img2 = cv2.resize(img2, resize)
            
        img1 = np.transpose(img1, (2, 0, 1))/256
        img2 = np.transpose(img2, (2, 0, 1))/256
        
        return img1, img2, np.array(label).reshape(1,)

    
train_folders = folders = glob(os.path.join("watermark/A_classification/train" , "*"))
val_folders = folders = glob(os.path.join("watermark/A_one_shot_recognition" , "*"))

train_dataset = CustomImageDataset(train_folders, 10000, augment = True)
val_dataset = CustomImageDataset(val_folders, 1000, augment = True)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [9]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.0 MB/s eta 0:00:00


In [10]:
import timm 
import torch
from torch import nn

In [11]:
class SeameseNetwork(nn.Module):
    def __init__(self, in_channels, image_shape_cfirst):
        super(SeameseNetwork, self).__init__()
        self.base_model = timm.create_model('efficientnet_b0', num_classes=100) #( in_channels, image_shape_cfirst)
        
    def forward(self, img1, img2):
        o1 = self.base_model(img1)
        o2 = self.base_model(img2)
        
        dist = euclidean_distance([o1, o2])
        return dist
        

net =  SeameseNetwork(1, (1, 112, 112))
net(torch.zeros(2, 3, 400, 400).float(), torch.zeros(2, 3, 400, 400).float()).shape

torch.Size([2, 1])

In [12]:
class AverageMeter:
    def __init__(self, name = None):
        if name is not None:
            self.name = name
        self.count = 0
        self.elements = []
        
    def update(self, val):
        self.count += 1
        self.elements.append(val)
        
    def avg(self):
        return sum(self.elements)/self.count
    
    def reset(self):
        self.count = 0
        self.elements = []

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class Trainer:
    def __init__(self, train_loader, val_loader):
        self.train_loader = train_loader
        self.val_loader = val_loader
#         self.test_loader = test_loader
        self.epoch = 0
        self.model = SeameseNetwork(3, (3, 400, 400)).to(device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.00001, betas=(0.9, 0.999))
        self.epoch_train_loss = []
        self.epoch_val_loss = []
        self.epoch_train_acc = []
        self.epoch_val_acc = []
        
        self.lossmeter = AverageMeter('Loss')
        self.avgmeter = AverageMeter('Average')
        self.avgmeterp2 = AverageMeter('AverageP2')
        self.avgmeterp1 = AverageMeter('AverageP1')
        
        
    def train_loop(self, epoch): 
        self.model.train()
        self.lossmeter.reset()
        self.avgmeter.reset()
        self.avgmeterp2.reset()
        self.avgmeterp1.reset()
        
        pbar = tqdm(self.train_loader, desc = 'Train: ', leave=False)
        # train loop
        for i, (x1, x2, y) in enumerate(pbar):
            x1 = x1.float().to(device)
            x2 = x2.float().to(device)   
            y = y.to(device)

            y_pred = self.model(x1, x2)

            loss = contrastive_loss(y, y_pred)
            self.lossmeter.update(loss.item())
            # optimize process
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            accuracy = compute_accuracy(y_pred, y, thr = 0.5)
#             accuracy_p2 = compute_accuracy(y_pred, y, thr = 0.2)
#             accuracy_p1 = compute_accuracy(y_pred, y, thr = 0.01)
            
            
            self.avgmeter.update(accuracy.item())
#             self.avgmeterp2.update(accuracy_p2.item())
#             self.avgmeterp1.update(accuracy_p1.item())
            
            pbar.set_postfix(epoch = f"{epoch+1}",
                            train_loss=self.lossmeter.avg(),
                            train_acc_p5 =self.avgmeter.avg(),
#                             train_acc_p2 =self.avgmeterp2.avg(),
#                             train_acc_p1 =self.avgmeterp1.avg(),
                            refresh=True)
            pbar.update(1)
            
        return self.lossmeter.avg(), self.avgmeter.avg()
                
    def val_loop(self, epoch):  
        self.model.eval()
        self.lossmeter.reset()
        self.avgmeter.reset()
#         self.avgmeterp2.reset()
#         self.avgmeterp1.reset()
        pbar = tqdm(self.val_loader, desc = 'Val: ', leave=False)
        # val loop
        for i, (x1, x2, y) in enumerate(pbar):
            x1 = x1.float().to(device)
            x2 = x2.float().to(device)   
            y = y.to(device)

            y_pred = self.model(x1, x2)

            loss = contrastive_loss(y, y_pred)
            self.lossmeter.update(loss.item())
            
            accuracy = compute_accuracy(y_pred, y, thr = 0.5)
#             accuracy_p2 = compute_accuracy(y_pred, y, thr = 0.2)
#             accuracy_p1 = compute_accuracy(y_pred, y, thr = 0.01)
            
            
            self.avgmeter.update(accuracy.item())
#             self.avgmeterp2.update(accuracy_p2.item())
#             self.avgmeterp1.update(accuracy_p1.item())
            
            
            pbar.set_postfix(epoch = f"{epoch+1}",
                             val_loss=self.lossmeter.avg(),
                             val_acc_p5 =self.avgmeter.avg(),
#                              val_acc_p2 =self.avgmeterp2.avg(),
#                              val_acc_p1 =self.avgmeterp1.avg(),
                             refresh=True)
            pbar.update(1)
        return self.lossmeter.avg(), self.avgmeter.avg()
    
    def one_shot_test(self, augment = False):
        self.model.eval()
        self.lossmeter.reset()
        self.avgmeter.reset()
        
        self.test_folder = 'watermark/A_one_shot_recognition'
        self.support_images = glob(os.path.join(self.test_folder , '*', "0.jpg"))
        self.target_images = glob(os.path.join(self.test_folder , '*', "1.jpg")) + glob(os.path.join(self.test_folder , '*', "2.jpg"))
        
        get_folder = lambda x : x.split('/')[-2]
        
        pbar = tqdm(self.target_images)
        for i, target_image in enumerate(pbar):
            target_folder = get_folder(target_image)
            target_img = cv2.imread(target_image)
            if augment:
                target_img = Preprocess(target_img)
                
            target_img = np.transpose(target_img, (2, 0, 1))/256
            
            best_score = 10000
            
            for support_image in tqdm(self.support_images, leave=False):
                support_img = cv2.imread(support_image)
                if augment:
                    support_img = Preprocess(support_img)
                support_img = np.transpose(support_img, (2, 0, 1))/256
                
                
                x1 = torch.unsqueeze(torch.from_numpy(target_img).float(), 0).to(device)
                x2 = torch.unsqueeze(torch.from_numpy(support_img).float(), 0).to(device)   
                
                y_pred = self.model(x1, x2).item()
                
                if y_pred < best_score:
                    best_score = y_pred
                    support_folder = get_folder(support_image)
                    
            if support_folder == target_folder:
                self.avgmeter.update(1)
            else:
                self.avgmeter.update(0)
                
            pbar.set_postfix(Current_Image = i,
                             Model_Accuracy = self.avgmeter.avg())
            
#         for i, (x1, x2, y) in enumerate(tqdm(self.test_loader)):
#             x1 = x1.float().to(device)
#             x2 = x2.float().to(device)   
#             y = y.to(device)

#             y_pred = self.model(x1, x2)

# #             loss = contrastive_loss(y, y_pred)
# #             self.lossmeter.update(loss.item())
            
#             accuracy = compute_accuracy(y_pred, y, thr = 0.5)
            
            
#             self.avgmeter.update(accuracy.item())
#             if i%10 == 9:
#                 print(f'ITER {i+1}: Model Accuracy is {self.avgmeter.avg()}')
                
    def run_training(self, n_epochs):
        main_pbar = tqdm(range(self.epoch, n_epochs + self.epoch ), desc = 'Epoch: ')
        for epoch in main_pbar:
            
            train_loss, train_acc = self.train_loop(epoch)
            self.epoch_train_loss.append(train_loss)
            self.epoch_train_acc.append(train_acc)
            
            val_loss, val_acc = self.val_loop(epoch)
            self.epoch_val_loss.append(val_loss)
            self.epoch_val_acc.append(val_acc)
            
            main_pbar.set_postfix(train_loss=self.epoch_train_loss[-1],
                                  train_acc =self.epoch_train_acc[-1],
                                  val_loss=self.epoch_val_loss[-1],
                                  val_acc =self.epoch_val_acc[-1],
                                  refresh=True)
            self.epoch += 1
            main_pbar.update(1)
            
    def plot_loss(self):
        plt.plot(self.epoch_train_loss)
        plt.plot(self.epoch_val_loss)
        plt.legend(['Train Loss','Valid Loss'])
        plt.show()
        
    def plot_acc(self):
        plt.plot(self.epoch_train_acc)
        plt.plot(self.epoch_val_acc)
        plt.legend(['Train Acc','Valid Acc'])
        plt.show()
        
    def save(self):
        torch.save({
            'epoch': self.epoch,
            'model_state_dict': trainer.model.state_dict(),
            'optimizer_state_dict': trainer.optimizer.state_dict(),
            }, f'seamese_model_epoch_{self.epoch}.pth')
        
    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.epoch = checkpoint['epoch']

trainer = Trainer(train_loader, val_loader)

In [23]:
trainer.run_training(50)

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Train:   0%|          | 0/1250 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [14]:
! gdown "1koUpLL5megOWUidnOD_7oMGxoLO9jVhq&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1koUpLL5megOWUidnOD_7oMGxoLO9jVhq&confirm=t
To: /content/seamese_model_epoch_50.pth
100% 50.0M/50.0M [00:00<00:00, 238MB/s]


In [16]:
# trainer.save()
trainer.load('seamese_model_epoch_50.pth')
trainer.one_shot_test(augment = True)

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: ignored